# Tensorflow Object Detection for Drone
## Detect Drone with Webcam using OpenCV
### Setup paths and directories
The first step is to make variables for all of the paths and directories.

In [1]:
import os

MODELS_DIR = os.path.join(os.getcwd(), 'exported-models')    # change this to the folder with the exported model

In [2]:
MODEL_NAME = 'my_ssd_mobilenet_v2_fpnlite'    # change this to the folder name of where the model is located
PATH_TO_CKPT = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'checkpoint/'))   
PATH_TO_CFG = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, 'pipeline.config'))

In [3]:
LABEL_FILENAME = 'drone_label_map.pbtxt'    # change this to whatever you named the label map file
PATH_TO_LABELS = os.path.join(MODELS_DIR, os.path.join(MODEL_NAME, LABEL_FILENAME))    # the label map should be located in the same folder with the checkpoints and saved model folders

### Load the Model
The next step is to load the model.

In [4]:
os.environ['TP_CPP_MIN_LOG_LEVEL'] = '2'    # supress TensorFlow logging
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

tf.get_logger().setLevel('ERROR')    # suppress TensorFlow logging (2)

C:\Users\timjr\AppData\Roaming\Python\Python37\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\timjr\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\timjr\AppData\Roaming\Python\Python37\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [5]:
# Enable GPU dynamic memory allocation
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [6]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(PATH_TO_CFG)
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)

In [7]:
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(PATH_TO_CKPT, 'ckpt-0')).expect_partial()

In [8]:
@tf.function
def detect_fn(image):
    """Detect objects in an image."""
    
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    
    return detections, prediction_dict, tf.reshape(shapes, [-1])

### Load Label Map Data (For Plotting)
Label maps correspond index numbers to category names, so that when our convolution network predicts 5, we know that this corresponds to _airplane_. Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine.

In [9]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

### Define the video stream
OpenCV will be used to capture the video stream from the webcam.

In [10]:
import cv2
cap = cv2.VideoCapture(0)    # 0 means main webcam

### Putting Everything Together
Finally we will load an image, run it through the detection model, and visualize the detection results, including the keypoints.

Note - This will take some time (several seconds) the first time it is run due to `tf.function`'s trace-compilation.

Some things to try:
- Modify some of the input images and see if detection still works. Some simple things to try out here (just uncomment the relevant portions of code) include flipping the image horizontally, or converting to grayscale (note that we still expect the input image to have 3 channels).
- Print out detections[‘detection_boxes’] and try to match the box locations to the boxes in the image. Notice that coordinates are given in normalized form (i.e., in the interval [0, 1]).
- Set `min_score_thresh` to other values (between 0 and 1) to allow more detections in or to filter out more detections.

In [ ]:
import numpy as np

while True:
    # Read frame from camera
    status, image_np = cap.read()
    
    # Expand dimensions since model expects images to have shape: [1, None, None, 3]
    image_np_expanded = np.expand_dims(image_np, axis=0)
    
    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)
    
    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections, predictions_dict, shapes = detect_fn(input_tensor)
    
    label_id_offset = 1
    image_np_with_detections = image_np.copy()
    
    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'][0].numpy(),
        (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
        detections['detection_scores'][0].numpy(),
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=200,
        min_score_thresh=.30,
        agnostic_mode=False)
    
    # Display output
    cv2.imshow('Object Detection', cv2.resize(image_np_with_detections, (800, 600)))
    
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
    
cap.release()
cv2.destroyAllWindows()